In [93]:
import pandas as pd
import numpy as np
from nltk import FreqDist, NaiveBayesClassifier

In [94]:
DATASET_PATH = "datasets/cleaned_dataset.csv"
SEED = 123123123

In [95]:
dataset = pd.read_csv(DATASET_PATH)
dataset.head()

,keywords,offensiveness_score
0,difference average earnings men women explaine...,-0.083
1,myth entirely based sex,-0.022
2,assertion women get paid less get paid less,-0.146
3,said op measuring women make jlaw complaining,-0.083
4,women payed less think many people say case th...,-0.042


In [96]:
def extract_features(keywords):
    return {keyword: True for keyword in keywords.split(" ")}


def extract_label(offensiveness_score):
    return "offensive" if offensiveness_score > 0 else "not_offensive"


In [97]:
train = dataset.sample(frac=0.8, random_state=123)
test = dataset.drop(train.index)
print(f"train samples: {len(train)}, test samples: {len(test)}")

train samples: 4542, test samples: 1135


In [98]:
x_train = [extract_features(keywords) for keywords in train["keywords"]]
y_train = [extract_label(score) for score in train["offensiveness_score"]]
classifier = NaiveBayesClassifier.train(zip(x_train, y_train))

In [144]:
classifier.most_informative_features()
text = "have a nice day sir" # quicktest
features = extract_features(text)
result = classifier.prob_classify(features)
for label in classifier.labels():
    print(f"{label}: {result.prob(label)}")

not_offensive: 0.8167424155902082
offensive: 0.18325758440979323
